In [1]:
import json
import random

# Set the number of random entries to select
num_entries = 9501

# Read the original dataset
with open('seeclick_web_train.json', 'r') as file:
    data = json.load(file)

# Select random entries
random_entries = random.sample(data, num_entries)

json_elements = []

for entry in random_entries:
    json_object = {
        'id': entry['id'],
        'conversations': entry['conversations']
    }
    json_elements.append(json_object)

# Convert the json_elements list to a JSON string
json_string = json.dumps(json_elements)

# Write the JSON string to a file
with open('small_train.json', 'w', encoding='utf-8') as file:
    json.dump(json_elements, file, ensure_ascii=False, indent=4)

FileNotFoundError: [Errno 2] No such file or directory: 'seeclick_web_train.json'

In [19]:
k = 2
with open(f"k_{k}_data.json", "r") as file:
    data = json.load(file)

with open("json_data/small_train.json", "r") as file:
    indexes = json.load(file)

# Updated processing to include both original and augmented data points with images in the prompt
small_data = []

for i, data_index in enumerate(indexes):
    index = int(data_index['id'].split("_")[1])
    for j in range(k):
        quadrant_item = data[index*(k + 1) + j]
        quadrant_item['id'] = f"identity_{i}_{j}"
        small_data.append(quadrant_item)
        
    coordinate_item = data[index*(k + 1) + k]
    coordinate_item['id'] = f"identity_{i}_{k}"
    small_data.append(coordinate_item)

# Save the updated augmented data to a new JSON file
augmented_file_path = f"json_data/small_k_{k}_train.json"
with open(augmented_file_path, "w") as file:
    json.dump(small_data, file, indent=4)
# create augmented data where one data 

In [23]:
import pandas as pd

# Load the CSV file to examine its structure
csv_file_path = 'data-recursive-main-fixed.csv'
data = pd.read_csv(csv_file_path)
data.head()

,Unnamed: 0,img_filename,new_name,instruction,quadrant,point,k
0,0,b1072ebd95d94a714a204ad0a44c2cb0.png,b1072ebd95d94a714a204ad0a44c2cb0-crop1row0.png,Composers -- Western Australia.,3,"(0.157552083333333, 0.8166666666666665)",1
1,1,b1072ebd95d94a714a204ad0a44c2cb0.png,b1072ebd95d94a714a204ad0a44c2cb0-crop2row0.png,Composers -- Western Australia.,3,"(0.315104166666666, 0.6333333333333331)",2
2,2,b1072ebd95d94a714a204ad0a44c2cb0.png,b1072ebd95d94a714a204ad0a44c2cb0-crop3row0.png,Composers -- Western Australia.,1,"(0.630208333333332, 0.2666666666666662)",3
3,3,b1072ebd95d94a714a204ad0a44c2cb0.png,b1072ebd95d94a714a204ad0a44c2cb0-crop1row1.png,InterLibrary Loan,1,"(0.74140625, 0.2907407407407405)",1
4,4,b1072ebd95d94a714a204ad0a44c2cb0.png,b1072ebd95d94a714a204ad0a44c2cb0-crop2row1.png,InterLibrary Loan,3,"(0.4828125, 0.581481481481481)",2


In [24]:
# Adjust the JSON format to include two entries for quadrant and one for position per image sequence
data_k1 = data[data['k'] == 1].copy()
data_k2 = data[data['k'] == 2].copy()
data_k3 = data[data['k'] == 3].copy()

# Merge the data for k=1, 3
merged_data = pd.merge(data_k1, data_k2, on=['img_filename', 'instruction'], suffixes=('_k1', '_k2',))
print(len(merged_data))
merged_data = pd.merge(merged_data, data_k3, on=['img_filename', 'instruction'], suffixes=('', '_k3'))
print(len(merged_data))

155304
780930


In [10]:
import pandas as pd

K = 2

csv_file_path = '../data-recursive-main.csv'
data = pd.read_csv(csv_file_path)
data.head()

datas = []
for k in range(1, K + 2):
    data_k = data[data['k'] == k].copy()
    datas.append(data_k)

merged_data = datas[0]
for k in range(1, K + 1):
    merged_data = pd.merge(merged_data, datas[k], on=['img_filename', 'instruction'], suffixes=(f'_{k-1}', f'_{k}'))

In [11]:
merged_data.head()

,Unnamed: 0_0,img_filename,new_name_0,instruction,quadrant_0,point_0,k_0,Unnamed: 0_1,new_name_1,quadrant_1,point_1,k_1,Unnamed: 0,new_name,quadrant,point,k
0,0,b1072ebd95d94a714a204ad0a44c2cb0.png,b1072ebd95d94a714a204ad0a44c2cb0-crop1row0.png,Composers -- Western Australia.,3,"(0.157552083333333, 0.8166666666666665)",1,1,b1072ebd95d94a714a204ad0a44c2cb0-crop2row0.png,3,"(0.315104166666666, 0.6333333333333331)",2,2,b1072ebd95d94a714a204ad0a44c2cb0-crop3row0.png,1,"(0.630208333333332, 0.2666666666666662)",3
1,3,b1072ebd95d94a714a204ad0a44c2cb0.png,b1072ebd95d94a714a204ad0a44c2cb0-crop1row1.png,InterLibrary Loan,1,"(0.74140625, 0.2907407407407405)",1,4,b1072ebd95d94a714a204ad0a44c2cb0-crop2row1.png,3,"(0.4828125, 0.581481481481481)",2,5,b1072ebd95d94a714a204ad0a44c2cb0-crop3row1.png,1,"(0.965625, 0.16296296296296212)",3
2,6,b1072ebd95d94a714a204ad0a44c2cb0.png,b1072ebd95d94a714a204ad0a44c2cb0-crop1row2.png,Bookmark link for this record,1,"(0.5882812499999995, 0.30000000000000004)",1,7,b1072ebd95d94a714a204ad0a44c2cb0-crop2row2.png,3,"(0.17656249999999904, 0.6000000000000001)",2,8,b1072ebd95d94a714a204ad0a44c2cb0-crop3row2.png,2,"(0.3531249999999981, 0.2000000000000002)",3
3,9,b1072ebd95d94a714a204ad0a44c2cb0.png,b1072ebd95d94a714a204ad0a44c2cb0-crop1row3.png,Author,1,"(0.889583333333333, 0.2240740740740735)",1,10,b1072ebd95d94a714a204ad0a44c2cb0-crop2row3.png,1,"(0.779166666666666, 0.448148148148147)",2,11,b1072ebd95d94a714a204ad0a44c2cb0-crop3row3.png,4,"(0.5583333333333319, 0.896296296296294)",3
4,12,b1072ebd95d94a714a204ad0a44c2cb0.png,b1072ebd95d94a714a204ad0a44c2cb0-crop1row4.png,Classic Catalogue Home,2,"(0.05625, 0.1212962962962955)",1,13,b1072ebd95d94a714a204ad0a44c2cb0-crop2row4.png,2,"(0.1125, 0.242592592592591)",2,14,b1072ebd95d94a714a204ad0a44c2cb0-crop3row4.png,2,"(0.225, 0.485185185185182)",3


In [4]:
print(len(merged_data))

155304


In [13]:
merged_data.head()

,Unnamed: 0_k1,img_filename,new_name_k1,instruction,quadrant_k1,point_k1,k_k1,Unnamed: 0_k2,new_name_k2,quadrant_k2,point_k2,k_k2,Unnamed: 0,new_name,quadrant,point,k
0,0,b1072ebd95d94a714a204ad0a44c2cb0.png,b1072ebd95d94a714a204ad0a44c2cb0-crop1row0.png,Composers -- Western Australia.,3,"(0.157552083333333, 0.8166666666666665)",1,1,b1072ebd95d94a714a204ad0a44c2cb0-crop2row0.png,3,"(0.315104166666666, 0.6333333333333331)",2,2,b1072ebd95d94a714a204ad0a44c2cb0-crop3row0.png,1,"(0.630208333333332, 0.2666666666666662)",3
1,3,b1072ebd95d94a714a204ad0a44c2cb0.png,b1072ebd95d94a714a204ad0a44c2cb0-crop1row1.png,InterLibrary Loan,1,"(0.74140625, 0.2907407407407405)",1,4,b1072ebd95d94a714a204ad0a44c2cb0-crop2row1.png,3,"(0.4828125, 0.581481481481481)",2,5,b1072ebd95d94a714a204ad0a44c2cb0-crop3row1.png,1,"(0.965625, 0.16296296296296212)",3
2,6,b1072ebd95d94a714a204ad0a44c2cb0.png,b1072ebd95d94a714a204ad0a44c2cb0-crop1row2.png,Bookmark link for this record,1,"(0.5882812499999995, 0.30000000000000004)",1,7,b1072ebd95d94a714a204ad0a44c2cb0-crop2row2.png,3,"(0.17656249999999904, 0.6000000000000001)",2,8,b1072ebd95d94a714a204ad0a44c2cb0-crop3row2.png,2,"(0.3531249999999981, 0.2000000000000002)",3
3,9,b1072ebd95d94a714a204ad0a44c2cb0.png,b1072ebd95d94a714a204ad0a44c2cb0-crop1row3.png,Author,1,"(0.889583333333333, 0.2240740740740735)",1,10,b1072ebd95d94a714a204ad0a44c2cb0-crop2row3.png,1,"(0.779166666666666, 0.448148148148147)",2,11,b1072ebd95d94a714a204ad0a44c2cb0-crop3row3.png,4,"(0.5583333333333319, 0.896296296296294)",3
4,12,b1072ebd95d94a714a204ad0a44c2cb0.png,b1072ebd95d94a714a204ad0a44c2cb0-crop1row4.png,Classic Catalogue Home,2,"(0.05625, 0.1212962962962955)",1,13,b1072ebd95d94a714a204ad0a44c2cb0-crop2row4.png,2,"(0.1125, 0.242592592592591)",2,14,b1072ebd95d94a714a204ad0a44c2cb0-crop3row4.png,2,"(0.225, 0.485185185185182)",3


In [15]:
import json

In [20]:


# Reinitialize the JSON data list
json_data = []
for index, row in merged_data.iterrows():
    if row["instruction"] == "":
        continue
    # Entry for quadrant prediction using k=1 image
    entry1 = {
        "id": f"identity_{index}_0",
        "conversations": [
            {
                "from": "user",
                "value": f"Picture 1: <img>data/recursive_augmented_images/{row['new_name_k1']}</img>\n In this UI screenshot, what is the partition of the element corresponding to the command \"{row['instruction']}\" (with quadrant number)?"
            },
            {
                "from": "assistant",
                "value": str(row['quadrant_k1'])
            }
        ]
    }

    # Entry for quadrant prediction using k=2 image
    entry2 = {
        "id": f"identity_{index}_1",
        "conversations": [
            {
                "from": "user",
                "value": f"Picture 1: <img>data/recursive_augmented_images/{row['new_name_k2']}</img>\n In this UI screenshot, what is the partition of the element corresponding to the command \"{row['instruction']}\" (with quadrant number)?"
            },
            {
                "from": "assistant",
                "value": str(row['quadrant_k2'])
            }
        ]
    }

    x = float(row['point'].split(',')[0].split('(')[1])
    y = float(row['point'].split(',')[1].split(')')[0])
    rounded_x = round(x, 2)
    rounded_y = round(y, 2)

    # Entry for position prediction using k=2 image
    entry3 = {
        "id": f"identity_{index}_2",
        "conversations": [
            {
                "from": "user",
                "value": f"Picture 1: <img>data/recursive_augmented_images/{row['new_name']}</img>\n In this UI screenshot, what is the position of the element corresponding to the command \"{row['instruction']}\" (with point)?"
            },
            {
                "from": "assistant",
                "value": f"({rounded_x}, {rounded_y})",
            }
        ]
    }

    json_data.append(entry1)
    json_data.append(entry2)
    json_data.append(entry3)

# Save to a JSON file with the new format
json_output_path = 'k_2_data.json'
with open(json_output_path, 'w') as json_file:
    json.dump(json_data, json_file, indent=4)

json_output_path

'k_2_data.json'

In [17]:
K = 1
AUGMENTED_PER_POINT = 3

# Load the CSV file to examine its structure
csv_file_path = '../data-recursive-main.csv'
data = pd.read_csv(csv_file_path)

json_data = []
for i in range(0, len(data), AUGMENTED_PER_POINT):
    for k in range(K):
        row = data.iloc[i + k]
        json_object = {
            "id": f"identity_{i//AUGMENTED_PER_POINT}_{k}",
            "conversations": [
                {
                    "from": "user",
                    "value": f"Picture 1: <img>data/recursive_augmented_images/{row['new_name']}</img>\n In this UI screenshot, what is the partition of the element corresponding to the command \"{row['instruction']}\" (with quadrant number)?"
                },
                {
                    "from": "assistant",
                    "value": str(row['quadrant'])
                }
            ]
        }
        json_data.append(json_object)

    row = data.iloc[i + K]
    
    x = float(row['point'].split(',')[0].split('(')[1])
    y = float(row['point'].split(',')[1].split(')')[0])
    rounded_x = round(x, 2)
    rounded_y = round(y, 2)

    json_object = {
                    "id": f"identity_{i//AUGMENTED_PER_POINT}_{K}",
                    "conversations": [
                        {
                            "from": "user",
                            "value": f"Picture 1: <img>data/recursive_augmented_images/{row['new_name']}</img>\n In this UI screenshot, what is the position of the element corresponding to the command \"{row['instruction']}\" (with point)?"
                        },
                        {
                            "from": "assistant",
                            "value": f"({rounded_x}, {rounded_y})",
                        }
                    ]
                }
    json_data.append(json_object)

with open(f'k_{K}_data.json', 'w') as file:
    json.dump(json_data, file, indent=4)

In [19]:
# make a json list of 9501 random indices from 0 to 113141
import json
import random

# Set the number of random indices to select
num_entries = 9501

# Select random indices
random_entries = random.sample(range(112764), num_entries)

# Convert the random indices to a JSON string
json_string = json.dumps(random_entries)

# Write the JSON string to a file
with open('random_indices.json', 'w', encoding='utf-8') as file:
    json.dump(random_entries, file, ensure_ascii=False, indent=4)

In [23]:
import json
import random

# Load the JSON file containing the random indices
with open('../json_data/random_indices.json', 'r') as file:
    random_indices = json.load(file)

# make random_indices a set for faster lookup
random_indices = set(random_indices)

test_indices = set()
# make a new json list of 500 different random indices from 0 to 112764
num_entries = 500
for i in range(num_entries):
    print(i)
    ith_entry = random.randint(0, 112764)
    while ith_entry in random_indices or ith_entry in test_indices:
        ith_entry = random.randint(0, 112764)
    test_indices.add(ith_entry)

test_indices = list(test_indices)

# Convert the random indices to a JSON string
json_string = json.dumps(test_indices)

# Write the JSON string to a file
with open('test_indices.json', 'w', encoding='utf-8') as file:
    json.dump(test_indices, file, ensure_ascii=False, indent=4)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [25]:
import pandas as pd
import json

In [26]:
csv_file_path = 'seeclick_web_bbox.csv'
data = pd.read_csv(csv_file_path)

In [28]:
json_data = []
index = 0
for i in range(0, len(data)):
    row = data.iloc[i]
    if row['instruction'] == ' ' or row['instruction'] == '' or row['instruction'] == '.':
        continue
    bbox = "(" + row['bbox'].split('[')[1].split(']')[0] + ")"
    json_object = {
        "image": "./train_data/"+row['img_filename'],
        "command": row['instruction'],
        "bbox": bbox
    }

    json_data.append(json_object)

with open('seeclick_web_bbox.json', 'w') as file:
    json.dump(json_data, file, indent=4)

In [29]:
len(json_data)

112765